# Fireworks Safe Zones - Interactive Analysis

Interactive analysis and visualization of safe zones for fireworks in any city.

This notebook uses the **current API** with differentiated buffers (20-300m) for 39 obstacle categories.

**Supports any city worldwide via OpenStreetMap!**

## Setup

In [ ]:
import sys
sys.path.insert(0, '..')

import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt

from src.osm_loader import get_city_boundary, load_all_obstacles
from src.geometry_utils import create_forbidden_zone_with_custom_buffers
from src.zones_generator import create_zones_from_free_space, add_zone_metadata
from src.config import get_buffer_distances, get_category_configs, get_utm_crs_for_location, DEFAULT_MIN_ZONE_AREA_M2

# Configure city to analyze
CITY = "Yerevan, Armenia"  # Change to any city: "Prague, Czech Republic", "Tbilisi, Georgia", etc.

print("Modules loaded successfully")
print(f"City configured: {CITY}")

## Configuration

In [ ]:
# Analysis parameters
MIN_ZONE_AREA = DEFAULT_MIN_ZONE_AREA_M2  # 2000 m²

print("Configuration:")
print(f"  City: {CITY}")
print(f"  Minimum zone area: {MIN_ZONE_AREA:,} m²")

# Show buffer configuration
buffers = get_buffer_distances()
print(f"\nBuffer distances: {len(buffers)} categories")
print(f"  Range: {min(buffers.values()):.0f}m - {max(buffers.values()):.0f}m")

## Load Data

This may take 2-3 minutes on first run (OSMnx caches data).

In [ ]:
print(f"Loading boundary for '{CITY}'...")
boundary = get_city_boundary(CITY)
boundary_area_km2 = boundary.geometry.area.sum() / 1_000_000

print("Boundary loaded:")
print(f"  Area: {boundary_area_km2:.2f} km²")
print(f"  CRS: {boundary.crs}")

# Auto-detect UTM zone
print("\nDetecting UTM zone...")
TARGET_CRS = get_utm_crs_for_location(boundary)
print(f"  Using {TARGET_CRS} for calculations")

In [ ]:
print("Loading obstacles from OpenStreetMap...")
print("  (This may take 2-3 minutes)\n")

obstacles = load_all_obstacles(
    boundary,
    include_sensitive=True,
    split_sensitive=True
)

total_features = sum(len(gdf) for gdf in obstacles.values())
non_empty_cats = sum(1 for gdf in obstacles.values() if len(gdf) > 0)

print("\nObstacles loaded:")
print(f"  Total features: {total_features:,}")
print(f"  Active categories: {non_empty_cats}/{len(obstacles)}")

## Obstacle Statistics

In [ ]:
# Create statistics table
configs = get_category_configs()
stats = []

for name, gdf in obstacles.items():
    if len(gdf) > 0 and name in configs:
        stats.append({
            'category': name,
            'count': len(gdf),
            'buffer_m': configs[name]['buffer_m'],
            'description': configs[name]['description']
        })

stats_df = pd.DataFrame(stats).sort_values('count', ascending=False)

print("Top 15 obstacle categories by count:\n")
print(stats_df.head(15).to_string(index=False))

In [ ]:
# Visualize obstacles by buffer distance
buffer_groups = stats_df.groupby('buffer_m')['count'].sum().sort_index(ascending=False)

fig, ax = plt.subplots(figsize=(10, 6))
buffer_groups.plot(kind='barh', ax=ax, color='steelblue')
ax.set_xlabel('Number of Features', fontsize=12)
ax.set_ylabel('Buffer Distance (m)', fontsize=12)
ax.set_title('Obstacle Features by Buffer Distance', fontsize=14, fontweight='bold')
ax.grid(axis='x', alpha=0.3)

for i, v in enumerate(buffer_groups.values):
    ax.text(v + 500, i, f'{v:,}', va='center', fontsize=9)

plt.tight_layout()
plt.show()

## Generate Safe Zones

In [ ]:
print("Creating forbidden zone with differentiated buffers...")
buffers = get_buffer_distances()
forbidden_zone, target_crs = create_forbidden_zone_with_custom_buffers(obstacles, TARGET_CRS, buffers)

print("Forbidden zone created")

In [ ]:
print(f"Generating safe zones (min area: {MIN_ZONE_AREA:,} m²)...")

zones = create_zones_from_free_space(
    boundary,
    forbidden_zone,
    TARGET_CRS,
    min_zone_area_m2=MIN_ZONE_AREA
)

zones = add_zone_metadata(zones, boundary)

total_zones = len(zones)
safe_area_km2 = zones['area_m2'].sum() / 1_000_000
coverage_pct = (safe_area_km2 / boundary_area_km2) * 100

print("\nSafe zones generated:")
print(f"  Total zones: {total_zones}")
print(f"  Safe area: {safe_area_km2:.2f} km² ({coverage_pct:.1f}%)")
print(f"  Excluded: {boundary_area_km2 - safe_area_km2:.2f} km² ({100-coverage_pct:.1f}%)")

## Zone Analysis

In [ ]:
# Size distribution
size_counts = zones['size_class'].value_counts()

print("Size class distribution:\n")
for size_class in ['small', 'medium', 'large', 'very_large']:
    if size_class in size_counts.index:
        count = size_counts[size_class]
        pct = (count / total_zones) * 100
        subset_area = zones[zones['size_class'] == size_class]['area_m2'].sum() / 1e6
        print(f"  {size_class:12s}: {count:4d} zones ({pct:5.1f}%), {subset_area:6.2f} km²")

In [ ]:
# Area statistics
print("\nArea statistics (m²):")
print(f"  Min:    {zones['area_m2'].min():12,.1f}")
print(f"  Max:    {zones['area_m2'].max():12,.1f}")
print(f"  Mean:   {zones['area_m2'].mean():12,.1f}")
print(f"  Median: {zones['area_m2'].median():12,.1f}")

print("\nCompactness statistics (1.0 = perfect circle):")
print(f"  Min:    {zones['compactness'].min():.3f}")
print(f"  Max:    {zones['compactness'].max():.3f}")
print(f"  Mean:   {zones['compactness'].mean():.3f}")

In [ ]:
# Visualize size distributionfig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))# Count by size classsize_order = ['small', 'medium', 'large', 'very_large']size_counts_ordered = [size_counts.get(s, 0) for s in size_order]colors = ['#FFB84D', '#FFE66D', '#A0E87C', '#4CAF50']ax1.bar(size_order, size_counts_ordered, color=colors, alpha=0.7, edgecolor='darkgreen')ax1.set_ylabel('Number of Zones', fontsize=12)ax1.set_title('Zone Count by Size Class', fontsize=14, fontweight='bold')ax1.grid(axis='y', alpha=0.3)for i, v in enumerate(size_counts_ordered):    if v > 0:        ax1.text(i, v + 2, str(v), ha='center', fontsize=10, fontweight='bold')# Area histogramax2.hist(zones['area_m2'] / 10000, bins=50, color='steelblue', alpha=0.7, edgecolor='black')ax2.set_xlabel('Area (hectares)', fontsize=12)ax2.set_ylabel('Frequency', fontsize=12)ax2.set_title('Zone Area Distribution', fontsize=14, fontweight='bold')ax2.grid(axis='y', alpha=0.3)plt.tight_layout()plt.show()

## Map Visualization

In [ ]:
# Reproject to UTM for visualization
print("Preparing map data...")
boundary_utm = boundary.to_crs(TARGET_CRS)
zones_utm = zones.to_crs(TARGET_CRS)

# Sample buildings for context
buildings = obstacles.get('buildings', gpd.GeoDataFrame())
if len(buildings) > 0:
    sample_size = min(3000, len(buildings))
    buildings_sample = buildings.sample(sample_size).to_crs(TARGET_CRS)
else:
    buildings_sample = None

print("Creating map...")

In [ ]:
fig, ax = plt.subplots(figsize=(18, 14))# Plot boundaryboundary_utm.boundary.plot(ax=ax, color='black', linewidth=2.5, zorder=5)# Plot buildingsif buildings_sample is not None:    buildings_sample.plot(ax=ax, facecolor='lightgray', edgecolor='gray',                          alpha=0.3, linewidth=0.3, zorder=1)# Plot safe zones by size class# Color scheme matching KML/visualizations (orange → yellow → green)size_colors = {    'small': '#FFB84D',      # Orange (2-5k m²)    'medium': '#FFE66D',     # Yellow (5-12.5k m²)    'large': '#A0E87C',      # Light green (12.5-50k m²)    'very_large': '#4CAF50'  # Green (50k+ m²)}for size_class, color in size_colors.items():    subset = zones_utm[zones_utm['size_class'] == size_class]    if len(subset) > 0:        subset.plot(ax=ax, facecolor=color, edgecolor='darkgreen',                   alpha=0.6, linewidth=0.5, zorder=3)# Title and labelsax.set_title(    f'{CITY} - Fireworks Safe Zones\n'    f'{total_zones} zones covering {safe_area_km2:.2f} km² ({coverage_pct:.1f}% of city)',    fontsize=16, fontweight='bold', pad=15)ax.set_xlabel('X (UTM meters)', fontsize=12)ax.set_ylabel('Y (UTM meters)', fontsize=12)# Legendlegend_elements = [    Patch(facecolor='black', edgecolor='black', label='City Boundary'),    Patch(facecolor='lightgray', alpha=0.3, label='Buildings (sample)')]for size_class, color in size_colors.items():    count = len(zones[zones['size_class'] == size_class])    if count > 0:        legend_elements.append(            Patch(facecolor=color, alpha=0.6, edgecolor='darkgreen',                  label=f'{size_class}: {count} zones')        )ax.legend(handles=legend_elements, loc='upper right', fontsize=11,          framealpha=0.95, edgecolor='black')ax.grid(True, alpha=0.2, linestyle='--')ax.set_aspect('equal')plt.tight_layout()plt.show()

## Top Zones Analysis

In [ ]:
# Show top 10 largest zones
top_zones = zones.nlargest(10, 'area_m2')

print("Top 10 largest safe zones:\n")
for idx, row in top_zones.iterrows():
    area_ha = row['area_m2'] / 10_000
    centroid = row.geometry.to_crs('EPSG:4326').centroid
    print(f"  Zone #{row['zone_id']:3d}: {area_ha:7.2f} ha ({row['size_class']:10s}) "
          f"- {centroid.y:.5f}°N, {centroid.x:.5f}°E")

## Export Results

In [ ]:
from pathlib import Path
from src.exporters import export_zones_to_geojson, export_zones_to_csv

output_dir = Path('../data')
output_dir.mkdir(exist_ok=True)

geojson_path = export_zones_to_geojson(zones, output_dir, city_name=CITY, target_crs=TARGET_CRS)
csv_path = export_zones_to_csv(zones, output_dir)

print("Results exported:")
print(f"  GeoJSON: {geojson_path}")
print(f"  CSV: {csv_path}")

## Summary

In [ ]:
print("="*70)
print("ANALYSIS SUMMARY")
print("="*70)

print("\nTerritory:")
print(f"  City area:        {boundary_area_km2:.2f} km²")
print(f"  Safe area:        {safe_area_km2:.2f} km² ({coverage_pct:.1f}%)")
print(f"  Excluded:         {boundary_area_km2 - safe_area_km2:.2f} km² ({100-coverage_pct:.1f}%)")

print("\nObstacles:")
print(f"  Total features:   {total_features:,}")
print(f"  Categories:       {non_empty_cats} active (39 total)")
print(f"  Buffer range:     {min(buffers.values()):.0f}-{max(buffers.values()):.0f} meters")

print("\nSafe zones:")
print(f"  Total zones:      {total_zones}")
print(f"  Size range:       {zones['area_m2'].min()/10000:.2f} - {zones['area_m2'].max()/10000:.2f} ha")
print(f"  Average area:     {zones['area_m2'].mean()/10000:.2f} ha")
print(f"  Avg compactness:  {zones['compactness'].mean():.3f}")

print("\n" + "="*70)